In [1]:
import os
import pandas as pd
import scanpy as sc

sc.logging.print_header()

scanpy==1.9.1 anndata==0.8.0 umap==0.5.3 numpy==1.19.2 scipy==1.6.2 pandas==1.3.3 scikit-learn==0.23.2 statsmodels==0.11.1 pynndescent==0.5.5


/home/icb/yuge.ji/miniconda3/envs/py37_fa_base/lib/python3.7/site-packages/umap/__init__.py:11: ImportWarning: Tensorflow not installed; ParametricUMAP will be unavailable
  category=ImportWarning,


In [ ]:
import warnings
warnings.filterwarnings('ignore')

## scGen

In [4]:
base_path = '/storage/groups/ml01/workspace/yuge.ji/sc-pert'

In [7]:
Frangi21 = sc.read(f'{base_path}/datasets/Frangieh_2021.h5ad')
Frangi21

AnnData object with n_obs × n_vars = 126966 × 22097
    obs: 'library_preparation_protocol', 'condition', 'MOI', 'sgRNA', 'UMI_count', 'n_genes', 'leiden', 'perturbation_name', 'perturbation_type', 'perturbation_value', 'perturbation_unit'
    var: 'n_cells', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'MOI_colors', 'condition_colors', 'doi', 'hvg', 'leiden', 'leiden_colors', 'library_preparation_protocol_colors', 'neighbors', 'pca', 'perturbation_type_colors', 'perturbation_unit_colors', 'perturbation_value_colors', 'preprocessing_nb_link', 'protein_names', 'umap'
    obsm: 'X_pca', 'X_umap', 'protein_expression'
    varm: 'PCs'
    obsp: 'connectivities', 'distances'

In [8]:
Frangi21 = sc.read(f'{base_path}/scperturb/FrangiehIzar2021_RNA.h5ad')
Frangi21

AnnData object with n_obs × n_vars = 218331 × 23712
    obs: 'library_preparation_protocol', 'perturbation_2', 'MOI', 'sgRNA', 'UMI_count', 'guide_id', 'umap_x', 'umap_y', 'perturbation', 'tissue_type', 'cancer', 'disease', 'perturbation_type', 'celltype', 'organism', 'perturbation_type_2', 'nperts', 'ngenes', 'ncounts', 'percent_mito', 'percent_ribo'
    var: 'ensembl_id', 'ncounts', 'ncells'

## scVI

## scANVI (HLCA)

In [10]:
import torch
import scvi
print(scvi.__version__)

AttributeError: type object 'torch._C._onnx.TensorProtoDataType' has no attribute 'BFLOAT16'

In [ ]:
hlca_path = '/lustre/groups/ml01/workspace/hlca_lisa.sikkema_malte.luecken/HLCA_reproducibility/data/HLCA_core_h5ads/HLCA_v1.h5ad'
full_adata = sc.read(hlca_path)

# subset adata to 2k genes originally used by scanvi
adata = full_adata[:, full_adata.var_names[full_adata.var['highly_variable']]].copy()

# load pre-trained model
model = scvi.model.SCANVI.load('/home/icb/yuge.ji/projects/HLCA_reproducibility/notebooks/3_atlas_extension/scanvi_model/', adata)
model